# Extractive Model Pipeline

In [ ]:
from haystack.document_stores import FAISSDocumentStore
from haystack.nodes import DensePassageRetriever
from haystack.pipelines import Pipeline
from haystack.nodes import TransformersReader
from haystack.utils import print_answers

document_store = FAISSDocumentStore.load(index_path="document_store.faiss", config_path="document_store.json")

retriever = DensePassageRetriever(
    document_store=document_store,
    query_embedding_model="facebook/dpr-question_encoder-single-nq-base",
    passage_embedding_model="facebook/dpr-ctx_encoder-single-nq-base",
    use_gpu=True
)
reader = TransformersReader(model_name_or_path="../models/BERT/roberta-base-squad2-finetuned-NLB-QA-2042-full_combined", use_gpu=True)

In [ ]:
p_extractive = Pipeline()
p_extractive.add_node(component=retriever, name="Retriever", inputs=["Query"])
p_extractive.add_node(component=reader, name="Reader", inputs=["Retriever"])
res = p_extractive.run(
    query="When were the anticorruption policies and procedures revised?", params={"Reader": {"top_k": 5}}
)
print_answers(res, details="all")

# Generative Model Pipeline

In [ ]:
from haystack.nodes import Seq2SeqGenerator
from haystack.nodes.answer_generator.transformers import _BartEli5Converter

generator = Seq2SeqGenerator(model_name_or_path="../models/T5/t5-small-finetuned-squadv2-finetuned-NLB-QA-2042-full_combined", input_converter=_BartEli5Converter())

In [ ]:
p_generative = Pipeline()
p_generative.add_node(component=retriever, name="Retriever", inputs=["Query"])
p_generative.add_node(component=generator,
                      name="Generator", inputs=["Retriever"])
res = p_generative.run(
    query="When were the anticorruption policies and procedures revised?", params={"Generator": {"top_k": 5}}
)
print_answers(res, details="all")